In [1]:
# Function for generating histogram
# Import Libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mlt
import datetime 

In [2]:
# This is to have a broader function that can get the data in the first place. (Datetime as well.)
def getData(year, direct, ASC, start_day, days, cols,datatype):
    for i in range(0, days):
        day = i + start_day
        year_str = '0.' + str(year)
        # File 
        if (day < 10):
            holdstr = 'pton00' + str(day) + year_str 
        elif (day < 100):
            holdstr ='pton0' + str(day) + year_str
        else: 
            holdstr = 'pton' + str(day) + year_str
        path = direct + holdstr + ASC
        if (day == start_day):
            data = np.array(np.loadtxt(path, dtype = datatype, skiprows = 1, usecols = cols))
        else: 
            hold = np.array(np.loadtxt(path, dtype = datatype, skiprows = 1, usecols = cols))
            data = np.concatenate((data, hold))
    return data

In [3]:
# Puts in the data that you have and returns precip events, nonprecip events, and total precip per precip event
def preciphist(data, minute, length, prec_duration, nonprec_duration):
    precipevent = np.zeros(length)
    nonprecip = np.zeros(length)
    preciptotal = np.zeros(length) # This is not intensity, but total precip per corresponding event.
    # This is to represent indices for the corresponding array from above
    prec = 0 # This represents both the precip events and total precip from precip events indices.
    nonprec = 0
    k = 0
    holdprec = 0 # These next three variables will hold the values, as we try to expand to include variable duration! 
    holdnonprec = 0
    holdtotal = 0
    # 
    Total_precip = 0; # Total precipitation measured by instrument 
    while (k < minute):
        hold = data[k]
        # If there is no precipitation at a certain minute
        if (hold == 0):
            holdnonprec += 1
            # How to deal with previous precipitation event
            if (holdprec >= prec_duration):
                precipevent[prec] = holdprec
                preciptotal[prec] = holdtotal
                prec += 1
                holdprec = 0
                holdtotal = 0
                # How to deal with a short precip event between nonprecip events
            elif (holdprec < prec_duration):
                holdnonprec += holdprec
                holdprec = 0
            # How to deal with the last event (nonprecipitation)
            if (k == minute-1):
                    nonprec += 1
                    nonprecip[nonprec] = holdnonprec
        #if (k >= 1):
         #   holdbefore = data[k-1]
          #  if (hold != holdbefore and hold == 0 and holdbefore > 0):
           #     prec += 1
        
        # Condition where we have precipitation    
        if (hold > 0):
            Total_precip += hold
            holdprec += 1
            holdtotal += hold
            # How to deal with previous non-precipitation event 
            if (holdnonprec >= nonprec_duration):
                nonprecip[nonprec] = holdnonprec
                nonprec += 1
                holdnonprec = 0
            # How to deal with a short nonprecip event between precip events
            elif (holdnonprec < nonprec_duration):
                holdprec += holdnonprec
                holdnonprec = 0
            # How to deal with the last event (if it is precipitation)
            if (k == minute -1):
                    prec += 1
                    precipevent[prec] = holdprec
                    preciptotal[prec] = holdtotal
            # How to account for new precip events
            #if (k >= 1):
             #   holdbefore = data[k-1]
              #  # Don't iterate prec if it is the first precip event
               # if (hold != holdbefore and hold > 0 and holdbefore == 0):
                #    nonprec += 1
        k = k+1
    return precipevent, nonprecip, preciptotal, Total_precip

# Needs to have an end-case for both precipitation and non-precipitation!!!! 

In [5]:
def calc(data, what_bin):
    hist, bin_edge = np.histogram(data, bins = what_bin)
    log_y = np.log(hist)
    bin_val = np.zeros(len(what_bin)-1)
    for i in range(len(what_bin)-1):
        bin_val[i] = (what_bin[i] + what_bin[i+1])/2
    logx = np.log(bin_val)
    curve_fit = np.polyfit(logx, log_y,1)
    curve_fit[1] = np.exp(curve_fit[1])
    return curve_fit, bin_val

In [6]:
# Exponentials get from durations
def exp_dur(in_put, exclude, length):
    first_bin = np.zeros(length)
    for i in range(length):    
        first_bin[i] = np.percentile(in_put, i*(100/length))
    second_bin = np.unique(first_bin)
    stop1 = np.array([exclude])
    final_bin = np.setdiff1d(second_bin, stop1)
    curvefit, bin_vals = calc(in_put, final_bin)
    return curvefit, final_bin, bin_vals
    

In [7]:
# Exponentials get from intensity
def exponential_int(in_put, exclude, length):
    first_bin = np.zeros(length)
    for i in range(length):    
        first_bin[i] = np.round(np.percentile(in_put, i*(100/length)),2)
    second_bin = np.unique(first_bin)
    stop1 = np.array([exclude])
    final_bin = np.setdiff1d(second_bin, stop1)
    curvefit, bin_vals = calc(in_put, final_bin)
    return curvefit, final_bin, bin_vals

In [8]:
# Simple climatology model uses the data inputted to calculate exponentials 
def model_1(data, amount, howmuch):
    precip, nonprecip, preciptotal, totalprep = histplot.preciphist(data, len(data), amount, 1, 1)
    count = 0
    count1 = 0
    for i in range(0, len(precip)):
        hold = precip[i]
        hold1 = nonprecip[i]
        if hold > 0:
            count += 1
        if hold1 > 0:
            count1 += 1
    precip = precip[:count]
    nonprecip = nonprecip[:count1]
    preciptotal = preciptotal[:count]
    # Calculate intensities 
    inten = np.divide(preciptotal, precip)
    # Exponentials for precipitation duration 
    pre_exp,pre_f_bins, bin_vals = exponential(precip, 1)
    # Exponentials for nonprecipitation duration 
    non_pre, nonpre_f_bins, nonp_b_vals = exponential(nonprecip, 1)
    # Exponentials for Intensity of Precipitation duration
    intensity, inten_f_bins, inten_b_vals = exponential_int(inten, 0.01)
    # print(nonpre_f_bins)
    # Construct intervals and probabilities for precipitation durations
    Pre_prob = np.zeros(1)
    pre_bin_1 = counting(precip, pre_f_bins[0])
    Pre_prob[0] = pre_bin_1
    construct_1 = pre_exp[1]*np.exp(np.log(bin_vals)*pre_exp[0])
    Pre_prob = np.append(Pre_prob, construct_1)
    Pre_prob = Pre_prob/sum(Pre_prob)
    #print(Pre_prob)
    # Non_precipitation duration probs
    NonPre_prob = np.zeros(1)
    nonpre_bin_1 = counting(nonprecip, nonpre_f_bins[0])
    NonPre_prob[0] = nonpre_bin_1
    construct_2 = non_pre[1]*np.exp(np.log(nonp_b_vals)*non_pre[0])
    NonPre_prob = np.append(NonPre_prob, construct_2)
    NonPre_prob = NonPre_prob/sum(NonPre_prob)
    #print(NonPre_prob)
    # Intensity probs
    Intensity_prob = np.zeros(1)
    inten_bin_1 = counting(inten, inten_f_bins[0])
    Intensity_prob[0] = inten_bin_1
    construct_3 = intensity[1]*np.exp(np.log(inten_b_vals)*intensity[0])
    Intensity_prob = np.append(Intensity_prob, construct_3)
    Intensity_prob = Intensity_prob/sum(Intensity_prob)
    #print(Intensity_prob)
    # Precipitation intervals
    Pre_interval = [1]
    Pre_interval = np.append(Pre_interval, pre_f_bins)
    #print(Pre_interval)
    # NonPrecipitation intervals
    NonPre_interval = [1]
    NonPre_interval = np.append(NonPre_interval, nonpre_f_bins)
    #print(NonPre_interval)
    # Intensity intervals
    Intensity_interval = [0.01]
    Intensity_interval = np.append(Intensity_interval, inten_f_bins)
    #print(Intensity_interval)
    Model_outputs = 0;
    # Running the crude model
    for i in range(howmuch):
        minutes = 0;
        Model = np.zeros(len(data))
        start = np.random.randint(0, 1)
        while (len(data)-1 > minutes):
            # run probability of precipitation event duration      
            pre_index = np.random.choice(np.arange(len(Pre_prob)), p = Pre_prob)
            pre_duration = np.random.randint(Pre_interval[pre_index], Pre_interval[pre_index + 1]+1) #Histogram for these synthetic data, show that they are similar 
            # to what you put in. 
            # run probability of precipitation event intensity
            inten_index = np.random.choice(np.arange(len(Intensity_prob)), p = Intensity_prob)
            model_intensity = 0.01*np.random.randint(100*Intensity_interval[inten_index], 100*Intensity_interval[inten_index+1]+1)
            # run probability of non-precipitation event duration
            nonpre_index = np.random.choice(np.arange(len(NonPre_prob)), p = NonPre_prob) 
            nonpre_duration = np.random.randint(NonPre_interval[nonpre_index], NonPre_interval[nonpre_index + 1]+1)
            # For each of the synthetic data have a histogram and look at to see if it is similar to what we calcluated for observed data. 
            # Nonprecipitation starts first
            if start == 0: 
                if (len(data) > minutes + pre_duration + nonpre_duration):
                    for j in range(nonpre_duration):
                        Model[minutes] = 0
                        minutes += 1
                    for k in range(pre_duration):
                        Model[minutes] = model_intensity
                        minutes += 1
                else: 
                    if (len(data) < minutes + nonpre_duration):
                        for j in range(len(data) - minutes):
                            Model[minutes] = 0
                            minutes += 1
                    elif (len(data) < minutes + nonpre_duration + pre_duration):
                        for j in range(nonpre_duration):
                            Model[minutes] = 0
                            minutes += 1
                        for k in range(len(data) - minutes):
                            Model[minutes] = model_intensity
                            minutes +=1
            # Precipitation starts first
            if start == 1: 
                if (len(data) > minutes + pre_duration + nonpre_duration):
                    for k in range(pre_duration):
                        Model[minutes] = model_intensity
                        minutes += 1
                    for j in range(nonpre_duration):
                        Model[minutes] = 0
                        minutes += 1
                else: 
                    if (len(data) < minutes + pre_duration):
                        for k in range(len(data) - minutes):
                            Model[minutes] = model_intensity
                            minutes +=1
                    elif (len(data) < minutes + pre_duration + nonpre_duration):
                        for k in range(pre_duration):
                            Model[minutes] = model_intensity
                            minutes += 1
                        for j in range(len(data) - minutes):
                            Model[minutes] = 0
                            minutes += 1
        Model_outputs = Model
    return Model_outputs, data 

In [9]:
# Starts with a simple model accuracy, precip with precip. 
def mod_acc1(model, observed, pre_min):
    mod_accuracy = 0
    for i in range(len(observed)):
        mod = model[i]
        obs = observed[i]
        #if (mod == 0 and obs == 0):
            #mod_accuracy += 1
        if (mod > 0 and obs > 0):
            mod_accuracy += 1
        else: 
            mod_accuracy += 0
    mod_accuracy = 100*mod_accuracy/pre_min
    return mod_accuracy

In [10]:
# maintains exact match for precip (i.e, match both precipitation occurs and the intensity)
def mod_acc2(model, observed, pre_min):
    mod_accuracy = 0
    for i in range(len(observed)):
        mod = model[i]
        obs = observed[i]
        #if (mod == 0 and obs == 0):
           # mod_accuracy += 1
        if (mod > 0 and obs > 0 and mod == obs):
            mod_accuracy += 1
        else: 
            mod_accuracy += 0
    mod_accuracy = 100*mod_accuracy/pre_min
    return mod_accuracy

In [ ]:
# Running model 1, however many times, get average rainfall and precipitation accuracy
def runit1(data, length, min_length, howmany):
    average_precip = 0
    avg_precip_accur = 0
    for i in range(howmany):
        sample_output, compare_data = model_1(data, length, min_length)
        average_precip = average_precip + sum(sample_output)
        pre_accuracy = mod_acc1(sample_output, new_summer18, pre_min)
        avg_precip_accur = avg_precip_accur + pre_accuracy
    average_precip = average_precip/howmany
    avg_precip_accur = avg_precip_accur/howmany
    return average_precip, avg_precip_accur